# Module: Communicating Analysis Results
## Sprint: Visualising Data using Power BI
## Part 1: Hands-On Practise Solved

## Your role

You are an analyst in the AdventureWorks company in the Data Ops department, which is responsible for fulfilling internal data needs, from adding and integrating new data sources, writing ETLs, and also developing dashboards that other teams would utilize.

Since you are well-known across the company as the go-to guy for making amazing interactive dashboards, your colleague from the Sales team visits you. Recently, he has provided the Sales Director with the sales data analysis you did in the previous module (Spreadsheets sprint). However they have realized that this data would be something they would like to track overtime, and they need it to be updated automatically. For this reason, he is looking for a person to transfer this analysis into the dashboard.

## Data

As in one of the previous sprints, you will be using [AdventureWorks company data](https://console.cloud.google.com/bigquery?authuser=3&project=tc-da-1&ws=!1m4!1m3!3m2!1stc-da-1!2sadwentureworks_db), which is readily available on BigQuery. You have already analyzed part of this data while doing Spreadsheets Sprint, though you got more familiar with databases once you moved to SQL sprint, so the data is already familiar to you. In case you need this data, you can always refer to the [schema of this database](https://i0.wp.com/improveandrepeat.com/wp-content/uploads/2018/12/AdvWorksOLTPSchemaVisio.png?ssl=1).

As you may have already guessed from the Hands-On task intro, we will be using charts and graphs, which you have already done in the Spreadsheets sprint. Since we aim to make sure that you have the skills to create dashboards, we will be providing SQL queries in some cases. In other cases, you may need to connect directly to BigQuery tables and maybe blend them together. The main purpose of this Hand On task is not to get more practice writing SQL queries but to get more practice developing interactive dashboards, which would be easy to understand and draw insights from.

While creating this dashboard, imagine that data in [AdventureWorks db](https://console.cloud.google.com/bigquery?authuser=3&project=tc-da-1&ws=!1m4!1m3!3m2!1stc-da-1!2sadwentureworks_db) is being daily. General advice when constructing a dashboard is to always think about the user and how you would look into the dashboard yourself if you were the Director of Sales. How would you interact with the dashboard, and what fields would you like to filter?

If you experience issues with connecting data sources, [here](https://docs.google.com/document/d/1DWHOrplrgdYOuUmJsb5i7VHXnkyaMIXMOR7DwlAm7d8/edit#heading=h.61wommm1sne4) are some instructions for that.

##Monday

On Monday, you start with adding some main KPIs to the top of your dashboard: Orders Count, Total Sales ($), Average Order Value (AOV) and Avg. Days to Ship. For the last 2 KPIs, you will need to implement custom calculations.

You then create 2 additional charts showing monthly sales:

1. Monthly sales by year, month
2. Monthly sales by month, year over year

To do this, you start by connecting the `salesorderheader` table from [AdventureWorks database](https://console.cloud.google.com/bigquery?authuser=3&project=tc-da-1&ws=!1m5!1m4!4m3!1stc-da-1!2sadwentureworks_db!3ssalesorderheader). Make sure you use PowerBI Connector for this. Initially, you will need to authorize access to this particular database.For this make sure you use Turing College provided Google account. Once authorized, make sure you connect only the `salesorderheader` table. You will be provided with two options: connect live data or create extract. It is up to you which one you use, but make sure you understand what each implies.

Next, you create your KPI graphs as big numbers on top of the dashboard, followed by line charts for monthly sales. Of course, you include a date range filter at the top of the dashboard, so your users could filter all these graphs by their selected dates.

<br>

Your final result should look something like this:


<div><img src="https://i.imgur.com/MomPAK9.png"></div>

Solved: [Monday Dashboard](https://drive.google.com/file/d/1Bks2ksQY5nuDbETVrRNDamwft2r2_Ee4/view?usp=drive_link)


##Tuesday

On Tuesday, you decide to add a new graph depicting sales by type: Offline vs Online. You talk with your colleague from the sales department and find out that all the sales in `salesorderheader` table with no salesperson ID actually indicate online sales, while sales with salesperson ID provided depict Offline sales done by salespeople in your company.

To do this add column to your table using the following formula:


In [ ]:
= Table.AddColumn(salesorderheader_Table, "Sales Type", each if [SalesPersonID] > 0 then "Offline" else "Online")

Always double-check if your data works as expected and that formula provides the results you need. Your dashboard after this should look something like this:

<div><img src="https://i.imgur.com/WHQtgYZ.png"></div>

Solved: [Tuesday Dashboard](https://drive.google.com/file/d/1hNmV5COuiELPJzUsZHKSFRy0rJlpmkTF/view?usp=drive_link)

##Wednesday

On Wednesday your colleague from the Sales department says that the Sales Department is also very interested in the dynamics of sales reasons and their performance over time. You decide to utilize PowerB’sI advanced options functionality when connecting data sources. You know that sales reasons reside in a separate table called `salesreason`, so you decide to write a custom SQL query to pull this data. Also to reduce the strain of pulling all the data, you decide to aggregate it on a year-month basis.

Looking at the data, you realize it could be better. In the absolute majority of cases, the sales reason is not assigned. Also, you notice that in roughly 20% of orders, more than 1 reason is assigned to a sale. To mitigate this, you decide to pull only data which actually has some sale reasons assigned. Furthermore, since several reasons may be assigned, you choose to create a stacked bar chart which would indicate the most popular reasons captured for certain months over the year.

Your final query looks something like this:

In [ ]:
WITH sales_per_reason AS (
 SELECT
   DATE_TRUNC(OrderDate, MONTH) AS year_month,
   sales_reason.SalesReasonID,
   SUM(sales.TotalDue) AS sales_amount
 FROM
   `tc-da-1.adwentureworks_db.salesorderheader` AS sales
 INNER JOIN
   `tc-da-1.adwentureworks_db.salesorderheadersalesreason` AS sales_reason
 ON
   sales.SalesOrderID = sales_reason.salesOrderID
 GROUP BY 1,2
)
SELECT
 sales_per_reason.year_month,
 reason.Name AS sales_reason,
 sales_per_reason.sales_amount
FROM
 sales_per_reason
LEFT JOIN
 `tc-da-1.adwentureworks_db.salesreason` AS reason
ON
 sales_per_reason.SalesReasonID = reason.SalesReasonID

Your new chart should look like this:

<div><img src="https://i.imgur.com/Aolin8I.png"></div>

Solved: [Wednesday Dashboard](https://drive.google.com/file/d/18Xv2UCgzjKYGCHY7yMrCgzyaimHQoBHu/view?usp=drive_link)
##Thursday

On Thursday you decide to include some information about Salesperson performance, so Sales Manager could see who out of his employees do best. For this, you construct a bar chart showing the total number of sales per Salesperson. For this, you will need to create a Calculated Table with new parameters already aggregated on Salesperson. This table will need to include:

- Sales Person New - a calculated dimension which takes SalespersonID and transforms it to string value, if it is missing assigns “Online” as the string
- Sum Sales - a calculation of total sales per “Sales Person New” dimension
- Cumulative Total - cumulative total of your “Sum Sales” calculation
- Ranking - a calculation which assigns rank of calculated sales per “Sales Person New” dimension
- Total Sales - a calculation which shows total sales of your whole data source
- Cumulative Percent - calculation which takes “Cumulative Total” and divides it by “Total Sales”

For more details on how to create such a table, follow the instructions for creating a Pareto chart provided [here](https://powerbi.tips/2016/10/pareto-charting/).

Your end result should look something like this:

<div><img src="https://i.imgur.com/YQjQ81T.png"></div>

Solved: [Thursday Dashboard](https://drive.google.com/file/d/1o9YCk-BP-zwScxRmKtYRgF6ZyAU3ESWm/view?usp=drive_link)

##Friday

On Friday, you decide to finish your dashboard with some geographical analysis. You decide to write a custom query and utilize tables related to shipping address ID in order to pull necessary geographical parameters. You will add this map as a separate tab in your dashboard. Your result should look something like this:

<div><img src="https://i.imgur.com/4hK0enY.png"></div>

In order to create such a graph, you will need to set up a new BigQuery connection with a custom query.

In [ ]:
SELECT
      salesorderheader.*,
      province.stateprovincecode as ship_province,
      province.CountryRegionCode as country_code,
      province.name as country_state_name
FROM `tc-da-1.adwentureworks_db.salesorderheader` as salesorderheader
INNER JOIN
     `tc-da-1.adwentureworks_db.address` as address
    ON salesorderheader.ShipToAddressID = address.AddressID
INNER JOIN
     `tc-da-1.adwentureworks_db.stateprovince` as province
    ON address.stateprovinceid = province.stateprovinceid

To create such a graph, you will need to add state and country code to your original data from “salesorderheader”. Choose the “Map” type of visualization, use “ship_province” as your location parameter and add “Sum of TotalDue” as your bubble size.

Solved: [Friday Dashboard](https://drive.google.com/file/d/1Xu1FZIuBdElihvfN7uP4x3GEayuSqg96/view?usp=drive_link)